Import necessary library at start

In [4]:
!pip install tensorflow-hub

In [5]:
#file processing
import os
import shutil
import string

#Data Anylisis
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#ML Packages
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


print("Version: ", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.4.1
GPU is NOT AVAILABLE


Load Data

In [7]:
data_judul_berita = pd.read_csv("/content/Data-Judul-Berita-GABUNGAN.csv",encoding='latin1',
                                error_bad_lines=False) #nanti kasih link csv nya biar gausa upload tiap kali mau training
data_judul_berita.drop(columns=['tanggal_publikasi','jam_publikasi'],inplace=True)
data_judul_berita

,judul_berita,kategori
0,JPO TB Simatupang akan Dimodifikasi,BUMD
1,Petugas Sudin Gulkarmat Jakpus Bantu Bersihkan...,Lintas Kota
2,Fadjar Panjaitan Terpilih Jadi Ketua Kwarda Pr...,Pendidikan
3,Musrenbang Kelurahan Sukapura Prioritaskan Tig...,Pembangunan
4,DPRD Aceh Studi Komparasi ke DPRD DKI,Legislatif
...,...,...
25279,Penataan Terminal Kampung Rambutan Capai 80 Pe...,Jakarta Hari Ini
25280,Festival Olahraga Sepanjang Tahun Digelar di P...,Olahraga
25281,"Siang Hari, Sebagian Wilayah Jakarta Diguyur H...",Jakarta Hari Ini
25282,Pemkot Jakut Gelar Apel Razia Pekat,Jakarta Hari Ini


In [8]:
labels = data_judul_berita['kategori']
labels.value_counts()

Jakarta Hari Ini          6287
Lintas Kota               4031
Kesra                     2626
BUMD                      1590
Pemerintahan              1385
Kesehatan                 1319
Legislatif                1216
Ekonomi                   1174
Pembangunan               1094
Fokus                      704
Pendidikan                 635
Transportasi               601
Olahraga                   594
Wisata & Kuliner           566
Banjir                     550
Seni & Budaya              509
Pemberdayaan Perempuan     300
Hut 492 DKI Jakarta         27
Warga  Bicara               26
LENGGANG JAKARTA            25
Tokoh Masyarakat             6
LINTAS KOTA                  6
Jakarta in Spotlight         5
HUT DKI ke 476               2
Info Humas                   2
Gubernatorial Election       1
info pemilu 2014             1
Jakarta UpBeat               1
Jakarte Punye Cerite         1
Name: kategori, dtype: int64

Data Exploration

In [9]:
berita_features = data_judul_berita.copy()
berita_labels = berita_features.pop('kategori')
berita_features

,judul_berita
0,JPO TB Simatupang akan Dimodifikasi
1,Petugas Sudin Gulkarmat Jakpus Bantu Bersihkan...
2,Fadjar Panjaitan Terpilih Jadi Ketua Kwarda Pr...
3,Musrenbang Kelurahan Sukapura Prioritaskan Tig...
4,DPRD Aceh Studi Komparasi ke DPRD DKI
...,...
25279,Penataan Terminal Kampung Rambutan Capai 80 Pe...
25280,Festival Olahraga Sepanjang Tahun Digelar di P...
25281,"Siang Hari, Sebagian Wilayah Jakarta Diguyur H..."
25282,Pemkot Jakut Gelar Apel Razia Pekat


Bagi, shuffle data supaya overfitnya kemungkinannya kecil

In [10]:
vocab_size = 1000 
embedding_dim = 16 
max_length = 120 
trunc_type = 'post' 
padding_type = 'post' 
oov_tok = '<OOV>' 

In [11]:
#data validasi + train data
judul_val = berita_features[:350]
partial_train_judul = berita_features[350:]
kategori_val = berita_labels[:350]
partial_train_kategori= berita_labels[350:]

In [12]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(partial_train_judul)
word_index = tokenizer.word_index
                       

train_sequences = tokenizer.texts_to_sequences(partial_train_judul)
train_padded = pad_sequences(train_sequences, padding = padding_type, maxlen = max_length) # YOUR CODE HERE

In [13]:
validation_sequences = tokenizer.texts_to_sequences(judul_val) # YOUR CODE HERE
validation_padded = pad_sequences(validation_sequences, padding = padding_type, maxlen = max_length) # YOUR CODE HERE

label_tokenizer = Tokenizer() # YOUR CODE HERE
label_tokenizer.fit_on_texts(berita_labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(partial_train_kategori)) # YOUR CODE HERE
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(kategori_val)) # YOUR CODE HERE

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


Insert pre-trained model

In [16]:
import tensorflow_hub as hub
model = "https://tfhub.dev/google/nnlm-id-dim128-with-normalization/2"
hub_layer = hub.KerasLayer(model,input_shape=[],dtype=tf.string,trainable=True)
hub_layer(berita_features)
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(56, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))

model.summary()

ValueError: ignored

Design rest of model (sisa outputnya doang)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation = 'relu'),
    tf.keras.layers.Dense(4, activation = 'softmax') #4 Dulu
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 100       
Total params: 16,508
Trainable params: 16,508
Non-trainable params: 0
_________________________________________________________________


Latihan tuh model baru

In [ ]:
history = model.fit(train_padded, 
                    partial_train_judul, 
                    epochs = 30, 
                    validation_data = (validation_padded, validation_label_seq), 
                    verbose = 2)

Becus kaga modelnya? Y/N

In [ ]:
results = model.evaluate(judul_val,kategori_val)
print(results)

klu uda puas, sisa save, export ama convert. jangan lupa kasih ke cloud team

Save and export model into necessary deployment platform

In [ ]:
!mkdir -p saved_model
model.save('saved_model/Porject_Model_1')
saved_model_dir = #isi directory modelnya
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)